# **KAPE Live Response & Digital Forensic Imaging Tools**
### **Course:** Digital Forensics Lab

This notebook documents the configuration and usage of forensic triage and imaging tools commonly used in incident response. The goal is to demonstrate understanding of:

- Best practices for live response using **KAPE (Kroll Artifact Parser & Extractor)**  
- Appropriate KAPE targets/modules for volatile and non-volatile data collection  
- Overview and example usage of disk imaging tools including:
  - `dd`
  - `dcfldd`
  - `ewfacquire`
  - `aff4imager` (pmem suite)
- Integrity verification of collected evidence using hashing  
- Chain of custody documentation and signing  

This notebook focuses on methodology and command syntax rather than running the tools within this environment, as many require OS-level access.

## **KAPE Overview**

KAPE (Kroll Artifact Parser and Extractor) is a Windows triage tool used for rapid artifact collection and processing. It is commonly deployed in live response scenarios where time and stability of the system are critical.

KAPE operates using:
- **Targets** — identify which artifacts to collect  
- **Modules** — process or parse those artifacts using external tools  

KAPE is typically run from removable media to reduce contamination of the host.


## **Best Practices for Live Response**

During live response, the examiner must be careful not to alter system state more than necessary. The following best practices apply:

1. **Use trusted external storage** for running tools and saving data.  
2. **Collect volatile data first** (RAM, running processes, network connections, clipboard, etc.).  
3. **Document system time and state** before collection begins.  
4. **Capture system artifacts using pre-configured KAPE targets** to ensure consistency.  
5. **Hash collected data immediately** after acquisition.  
6. **Record every action in a chain of custody log.**

These practices support evidentiary integrity and admissibility.

## **KAPE Configuration for Live Response**

### **Recommended KAPE Targets for Incident Response**

The following targets are standard for live collection:

- `!SANS_Triage`  
- `KapeTriage`  
- `LiveResponse`  
- `RegistryHives`  
- `EventLogs`  
- `ProcessMemory`  
- `Network\*` (connections, interfaces, configurations)

These provide a comprehensive snapshot of system state with minimal footprint.

### **Recommended KAPE Modules**

KAPE modules allow processing of artifacts using parsers such as:

- **MFTECmd** – MFT parser  
- **EvtxECmd** – Windows Event Log parser  
- **RECmd** – Registry analysis  
- **BrowsingHistoryView** – browser artifacts  
- **SBECmd** – Shellbags  
- **PECmd** – Prefetch analysis  

Modules automate common forensic workflows and provide immediate insight.

---

## **Example KAPE Commands**

Below are example commands that would be executed from PowerShell:

`.\kape.exe --tsource C: --tdest E:\KAPE_Collection --target !SANS_Triage`

Collect artifacts and process them with modules:

`.\kape.exe --tsource C: --tdest E:\Output --target KapeTriage --module KapeModules`


Collect only registry and event logs:

`.\kape.exe --tsource C: --tdest E:\IR --target RegistryHives,EventLogs`

### Disk Imaging Tools Overview

This section explains common forensic imaging utilities and includes example usage.

##### `dd` Command

`dd` is a classic Unix utility used to create bit-for-bit copies of disks.

Example disk acquisition:

`dd if=/dev/sda of=/images/disk.dd bs=4M status=progress`

 ##### `dcfldd` Command

`dcfldd` is a forensic-enhanced version of dd, adding hashing, split output, and progress indicators.

Example with hashing:

`dcfldd if=/dev/sda of=disk.img hash=sha256 hashlog=disk_hash.txt`

##### `ewfacquire` Command

`ewfacquire` produces Expert Witness Format (E01) images commonly used in forensic suites.

Example acquisition:

`ewfacquire --compression-level=optimal --segment-size=2GiB /dev/sda`


This produces .E01 files along with metadata.

##### `AFF4Imager` (pmem suite)

`AFF4` is a modern flexible forensic format especially suited for memory capture.

Memory acquisition example:

`aff4imager --memory /evidence/memory.aff4`


Disk acquisition example:

`aff4imager --source /dev/sda --output /evidence/disk.aff4`

##### Integrity Verification

Hashing ensures that acquired evidence has not been altered. `SHA256` is the recommended standard.

Example Hashing Operations

Hash acquired image:

`sha256sum disk.dd > disk.dd.sha256`

Verify hash:

`sha256sum disk.dd`


The hash value must match the stored hash file to preserve integrity.